In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from numpy import loadtxt
from numpy import sort
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import bokeh
import sklearn
import xgboost as xgb
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.mlab as mlab
import sqlite3
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from numpy import loadtxt
from numpy import sort
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import bokeh
import sklearn
import xgboost as xgb
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.mlab as mlab
import sqlite3
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

#import packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import time 
import datetime
import collections
# from plotnine import *
import seaborn as sns
import scipy.stats as stats

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge 
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import pandas as pd


import xgboost as xgb
import lightgbm as lgb

This file is 35 column with no missing data 


123.csv : no missing value 

In [ ]:
zillow = pd.read_csv('/content/drive/MyDrive/SWE project/DataStorage/123.csv')

In [ ]:
zillow.shape

(88465, 35)

In [ ]:
zillow.isnull().sum()

logerror                        0
airconditioningtypeid           0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedbathnbr               0
finishedfloor1squarefeet        0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
finishedsquarefeet50            0
fips                            0
fireplacecnt                    0
fullbathcnt                     0
garagecarcnt                    0
garagetotalsqft                 0
heatingorsystemtypeid           0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertylandusetypeid           0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidneighborhood            0
regionidzip                     0
roomcnt                         0
threequarterbathnbr             0
unitcnt                         0
yearbuilt                       0
numberofstorie

Then we need to see each feature with target variable


In [ ]:
zillow.head()

,logerror,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet50,...,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock
0,0.0276,1.000000,2.0,3,4.0,2.0,1213.533333,1684.0,1684.0,1213.533333,...,0,1.0,1.000000,1959.0,1.266667,122754.0,360170.0,237416.0,6735.88,6.037110e+13
1,-0.1684,6.333333,3.5,4,5.6,3.5,1433.600000,2263.0,2263.0,1433.600000,...,0,1.0,1.066667,2014.0,1.533333,346458.0,585529.0,239071.0,10153.02,6.054730e+13
2,-0.0040,1.000000,3.0,2,4.0,3.0,1332.400000,2217.0,2217.0,1332.400000,...,0,1.0,1.000000,1940.0,1.266667,61994.0,119906.0,57912.0,11484.48,6.037460e+13
3,0.0218,1.000000,2.0,2,4.0,2.0,1360.866667,839.0,839.0,1360.866667,...,0,1.0,1.000000,1987.0,1.333333,171518.0,244880.0,73362.0,3048.74,6.037300e+13
4,-0.0050,8.466667,2.5,4,5.0,2.5,1188.533333,2283.0,2283.0,1188.533333,...,8,1.0,2.066667,1981.0,2.000000,169574.0,434551.0,264977.0,5488.96,6.059040e+13


Transform Transform incorrectly scaled variables

As mentioned in the data dictionary, the latitude and longitude has been multiplied to 10^6.

I will perform a division of the same to scale it to original.

I will perform a similar operation for 'rawcensustractandblock'.

Also, I will perform a division of 10^12 for the 'censustractandblock' variable to bring it to original scal

In [ ]:
zillow[['latitude', 'longitude']] = (zillow[['latitude', 'longitude']])/(10**6)

zillow['censustractandblock'] = (zillow['censustractandblock'])/(10**12)

zillow['rawcensustractandblock'] = (zillow['rawcensustractandblock'])/(10**6)


Checking Outliers & Removing them

In [ ]:
# Using Z-score as a threhold (WE can also find outliers using IQR and boxplot)
# Set thershold and 
z = np.abs(stats.zscore(zillow))
no_out_df = zillow[(z<3).all(axis=1)]
no_out_df.shape

(66882, 35)

In [ ]:
no_out_df.describe()

,logerror,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet50,...,roomcnt,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock
count,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,...,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,66882.000000,6.688200e+04,6.688200e+04,66882.000000,66882.000000
mean,0.005549,2.797103,2.171466,2.921803,5.202347,2.184924,1368.051442,1626.966810,1627.484151,1374.990299,...,0.861667,1.000152,1.216958,1968.778225,1.308630,153774.463813,3.908810e+05,2.375437e+05,5219.367979,60.433817
std,0.064786,3.172251,0.814473,0.974925,1.596161,0.796817,100.680070,667.898268,667.609869,107.932509,...,2.230220,0.003175,0.484162,23.271206,0.195689,110426.023603,2.834650e+05,2.133569e+05,3499.681793,0.096283
min,-0.239500,1.000000,0.000000,0.000000,1.000000,1.000000,848.133333,2.000000,2.000000,848.133333,...,0.000000,1.000000,1.000000,1899.000000,1.000000,100.000000,2.200000e+01,2.200000e+01,133.320000,60.371000
25%,-0.024300,1.000000,2.000000,2.000000,4.000000,2.000000,1306.000000,1156.000000,1157.000000,1307.200000,...,0.000000,1.000000,1.000000,1953.000000,1.266667,78900.000000,1.918245e+05,7.498700e+04,2840.760000,60.373000
50%,0.005000,1.000000,2.000000,3.000000,4.600000,2.000000,1364.133333,1479.000000,1479.000000,1369.266667,...,0.000000,1.000000,1.000000,1969.000000,1.266667,125390.000000,3.314930e+05,1.875070e+05,4470.160000,60.375700
75%,0.034400,3.933333,3.000000,4.000000,7.000000,3.000000,1422.933333,1943.000000,1943.000000,1434.000000,...,0.000000,1.000000,1.000000,1987.000000,1.333333,195700.000000,5.103080e+05,3.300000e+05,6621.082500,60.590200
max,0.259300,12.866667,5.000000,6.000000,10.000000,5.000000,2008.066667,4393.000000,4393.000000,2056.266667,...,9.000000,1.066667,3.666667,2015.000000,2.133333,786043.000000,2.058889e+06,1.434454e+06,25727.300000,60.710000


Store current stage data

In [ ]:
#we need to save data here
from google.colab import files
no_out_df.to_csv('dataBeforeFeatureEngineering.csv') 
files.download('dataBeforeFeatureEngineering.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
no_out_df.shape

(66882, 35)